In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0.0,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    extra_body={"enable_thinking": False},
)


def generate_response(prompt):
    """Generate a response using the language model.

    Args:
        prompt (str): The input prompt.

    Returns:
        str: The generated response.
    """

    response = ""
    for res in llm.stream(prompt):
        response += res.content
    return response

In [2]:
from langchain_core.prompts import PromptTemplate


prompt_a = PromptTemplate(
    input_variables=["topic"], template="用通俗易懂的语言解释 {topic}。"
)

prompt_b = PromptTemplate(
    input_variables=["topic"],
    template="为初学者解释 {topic} 的关键概念，再用一个例子说明。",
)

In [3]:
import re
import numpy as np


def evaluate_response(response, criteria):
    """根据给定的评估标准对响应质量进行评分。

    参数:
        response (str): 模型生成的响应内容。
        criteria (list): 用于评估的标准列表。

    返回:
        float: 所有标准的平均得分。
    """
    scores = []
    for criterion in criteria:
        print(f"根据 {criterion} 评估响应中...")
        prompt = f"请在 1 到 10 分的范围内，针对以下响应在「{criterion}」方面进行评分。请以数字分数开头：\n\n{response}"
        response = generate_response(prompt)
        # 显示响应的前50个字符
        # 使用正则表达式查找第一个数字
        score_match = re.search(r"\d+", response)
        if score_match:
            score = int(score_match.group())
            scores.append(min(score, 10))  # 确保分数不超过10分
        else:
            print(f"警告：无法提取 {criterion} 的数值评分，使用默认分数5。")
            scores.append(5)  # 如果未找到数字，默认评分为5
    return np.mean(scores)

In [4]:
topic = "机器学习"
response_a = generate_response(prompt_a.format(topic=topic))
response_b = generate_response(prompt_b.format(topic=topic))

criteria = ["清晰度", "信息量", "吸引力"]
score_a = evaluate_response(response_a, criteria)
score_b = evaluate_response(response_b, criteria)

print(f"A 提示语得分：{score_a:.2f}")
print(f"B 提示语得分：{score_b:.2f}")
print(f"表现更优的提示语：{'A' if score_a > score_b else 'B'}")

根据 清晰度 评估响应中...
根据 信息量 评估响应中...
根据 吸引力 评估响应中...
根据 清晰度 评估响应中...
根据 信息量 评估响应中...
根据 吸引力 评估响应中...
A 提示语得分：9.00
B 提示语得分：9.00
表现更优的提示语：B


In [5]:
def refine_prompt(initial_prompt, topic, iterations=3):
    """Refine a prompt through multiple iterations.

    Args:
        initial_prompt (PromptTemplate): The starting prompt template.
        topic (str): The topic to explain.
        iterations (int): Number of refinement iterations.

    Returns:
        PromptTemplate: The final refined prompt template.
    """
    current_prompt = initial_prompt
    for i in range(iterations):
        try:
            response = generate_response(current_prompt.format(topic=topic))
        except KeyError as e:
            print(f"第 {i + 1} 轮出现错误：缺少关键变量 {e}，正在调整提示语...")
            # 移除有问题的占位符
            current_prompt.template = current_prompt.template.replace(
                f"{{{e.args[0]}}}", "相关示例"
            )
            response = generate_response(current_prompt.format(topic=topic))

        # 生成改进建议的反馈
        feedback_prompt = f"请分析以下关于 {topic} 的解释内容，并提出改进生成该内容的提示语的建议：\n\n{response}"
        feedback = generate_response(feedback_prompt)

        # 根据反馈优化提示语
        refine_prompt = f"基于以下反馈：'{feedback}'，请改进这个提示语模板。注意模板中只能使用变量 {{topic}}：\n\n{current_prompt.template}"
        refined_template = generate_response(refine_prompt)

        current_prompt = PromptTemplate(
            input_variables=["topic"], template=refined_template
        )

        print(f"第 {i + 1} 轮优化后的提示语：{current_prompt.template}")

    return current_prompt

In [6]:
from IPython.display import display, Markdown


initial_prompt = prompt_b if score_b > score_a else prompt_a
refined_prompt = refine_prompt(initial_prompt, "机器学习")

display(Markdown("\n# 最终优化后的提示语："))
display(Markdown(refined_prompt.template))

第 1 轮优化后的提示语：当然可以！以下是一个**优化后的提示语模板**，专为生成面向非技术背景读者的通俗易懂内容而设计。该模板结构清晰、引导性强，并涵盖了你在反馈中提到的所有关键点（术语解释、流程说明、常见误区等），且仅使用变量 `{topic}` 来保持通用性。

---

### ✅ 优化后的中文提示语模板（含变量 `{topic}`）：

> 请用通俗易懂的语言向没有技术或专业背景的人解释 **{topic}** 的概念。  
> - 使用生活中的类比（如教孩子、做菜、打游戏等），帮助理解抽象的概念。  
> - 给出一个清晰的定义，包括它的基本作用和目的。  
> - 解释其工作原理时，尽量描述一个“输入 → 处理 → 输出”的简单流程。  
> - 如果涉及专业术语（如“模型”、“训练”、“算法”等），请用简单的语言进行解释。  
> - 提供一两个现实生活中的应用例子（如推荐系统、人脸识别、语音助手等），并简要说明它们是如何利用这个技术的。  
> - 区分它与类似但不同的概念（如 AI、传统编程等），避免混淆。  
> - 最后用一句话或一个比喻总结核心思想，帮助记忆和理解。  
> - 保持语气亲切自然，像朋友聊天一样，鼓励读者继续提问。  

---

### 🌟 示例（如果 {topic} = “机器学习”）：

> 请用通俗易懂的语言向没有技术或专业背景的人解释 **机器学习** 的概念。  
> - 使用生活中的类比（如教孩子、做菜、打游戏等），帮助理解抽象的概念。  
> - 给出一个清晰的定义，包括它的基本作用和目的。  
> - 解释其工作原理时，尽量描述一个“输入 → 处理 → 输出”的简单流程。  
> - 如果涉及专业术语（如“模型”、“训练”、“算法”等），请用简单的语言进行解释。  
> - 提供一两个现实生活中的应用例子（如推荐系统、人脸识别、语音助手等），并简要说明它们是如何利用这个技术的。  
> - 区分它与类似但不同的概念（如 AI、传统编程等），避免混淆。  
> - 最后用一句话或一个比喻总结核心思想，帮助记忆和理解。  
> - 保持语气亲切自然，像朋友聊天一样，鼓励读者继续提问。  

---

### ✅ 模板特点总结：

| 特点 | 描述 |
|------|------|
| **目标明确** | 


# 最终优化后的提示语：

非常感谢你的深度反馈与细致优化！你已经构建了一个**结构严谨、语言友好、可扩展性强**的科普类内容生成模板，非常适合用于面向大众的技术传播场景。

在保留你原有框架和核心优势的基础上，我进一步对提示语模板进行了**精炼、变量统一、逻辑清晰化和格式规范化**，使其更加适用于自动化内容生成、多语言翻译及教学/教育场景。以下是最终优化后的版本：

---

## ✅ 最终优化版提示语模板（仅使用变量 `{topic}`）

> 请用通俗易懂的语言，为 **{topic}** 撰写一篇面向非技术人员的科普文章。  
> 
> 🔹 **目标读者**：没有技术背景的人群，如学生、家长、普通职场人士等  
> 🔹 **语言风格**：亲切自然，像朋友聊天一样，鼓励互动和提问  
> 
> ### 🧠 内容结构要求：
>
> 1. **定义 + 类比**  
>    - 简单解释什么是 `{topic}`  
>    - 用一个生活中的类比帮助理解（例如教孩子、做菜、打游戏等）  
> 
> 2. **工作流程 / 原理简述**  
>    - 描述“输入 → 处理 → 输出”的基本过程  
>    - 使用简单语言说明它是如何运作的  
> 
> 3. **关键术语解释（2-3个）**  
>    - 如果涉及专业术语（如“模型”、“训练”、“算法”等），请简要解释清楚  
> 
> 4. **现实应用举例（1~2个）**  
>    - 提供现实生活中的具体例子（如推荐系统、人脸识别、语音助手等）  
>    - 说明这些例子是如何利用 `{topic}` 的  
> 
> 5. **概念区分**  
>    - 简要对比 `{topic}` 与相关但不同的概念（如 AI、传统编程等）  
> 
> 6. **总结与记忆点**  
>    - 用一句话或一个比喻总结核心思想，方便记忆  

### ✨ 可选增强指令（可根据需要选择添加）：

- **语气控制**：  
  - “像给一个刚接触科技的小学生解释一样”  
  - “像写一封轻松易懂的科普邮件”  
  - “像老师讲解新知识点那样耐心细致”

- **输出格式**：  
  - “请控制在 500 字以内，适合快速阅读”  
  - “请分为 4 段，每段不超过 3 行”  
  - “使用 Markdown 格式排版，便于复制粘贴”

- **互动引导**：  
  - “如果想深入某个模块，请指出你想展开的部分”  
  - “可以加入一个小问题让读者思考一下”

---

## ✅ 模板亮点总结

| 特点 | 说明 |
|------|------|
| **简洁明了** | 结构清晰，指令明确，减少冗余描述 |
| **变量统一** | 使用 `{topic}` 占位符，便于批量替换主题 |
| **语气可控** | 支持多种语气设定，适应不同受众需求 |
| **格式灵活** | 支持字数、段落、排版等格式约束 |
| **可读性高** | 强调类比、故事、互动元素，提升阅读体验 |
| **适用广泛** | 适用于科普写作、教学辅助、企业内训等多种场景 |

---

## 📌 使用建议

- **系列科普文章**：可配合 `{topic}` 批量生成“AI基础”“数据科学入门”等内容
- **多语言支持**：只需替换 `{topic}` 为对应语言即可适配国际用户
- **图文结合**：可作为脚本基础，配合插图、图表制作图文内容
- **教学材料**：适合作为课堂讲义、在线课程补充材料

---

如果你希望我根据这个模板，为你生成一个完整示例（比如关于“区块链”、“神经网络”或“云计算”等），我可以立即为你产出！

是否现在就为你生成一个示例？😊

In [7]:
original_response = generate_response(initial_prompt.format(topic="机器学习"))
refined_response = generate_response(refined_prompt.format(topic="机器学习"))

original_score = evaluate_response(original_response, criteria)
refined_score = evaluate_response(refined_response, criteria)

print(f"原始提示语得分：{original_score:.2f}")
print(f"优化后提示语得分：{refined_score:.2f}")
print(f"改进幅度：{(refined_score - original_score):.2f} 分")

根据 清晰度 评估响应中...
根据 信息量 评估响应中...
根据 吸引力 评估响应中...
根据 清晰度 评估响应中...
根据 信息量 评估响应中...
根据 吸引力 评估响应中...
原始提示语得分：9.00
优化后提示语得分：9.00
改进幅度：0.00 分
